In [ ]:
!pip install mtcnn

# Import Libraries

In [24]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
import tensorflow as tf
from glob import glob

In [25]:
import os

In [26]:
IMAGE_SIZE = [224, 224]


In [27]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [ ]:
!kaggle datasets download -d cashutosh/gender-classification-dataset

 95% 257M/269M [00:01<00:00, 135MB/s]
100% 269M/269M [00:01<00:00, 147MB/s]


In [ ]:
!unzip /content/gender-classification-dataset.zip -d ./

Streaming output truncated to the last 5000 lines.
  inflating: ./Validation/male/067031.jpg.jpg  
  inflating: ./Validation/male/067038.jpg.jpg  
  inflating: ./Validation/male/067040.jpg.jpg  
  inflating: ./Validation/male/067046.jpg.jpg  
  inflating: ./Validation/male/067048.jpg.jpg  
  inflating: ./Validation/male/067054.jpg.jpg  
  inflating: ./Validation/male/067064.jpg.jpg  
  inflating: ./Validation/male/067068.jpg.jpg  
  inflating: ./Validation/male/067069.jpg.jpg  
  inflating: ./Validation/male/067086.jpg.jpg  
  inflating: ./Validation/male/067091.jpg.jpg  
  inflating: ./Validation/male/067099.jpg.jpg  
  inflating: ./Validation/male/067101.jpg.jpg  
  inflating: ./Validation/male/067102.jpg.jpg  
  inflating: ./Validation/male/067112.jpg.jpg  
  inflating: ./Validation/male/067117.jpg.jpg  
  inflating: ./Validation/male/067119.jpg.jpg  
  inflating: ./Validation/male/067133.jpg.jpg  
  inflating: ./Validation/male/067143.jpg.jpg  
  inflating: ./Validation/male/067152

In [ ]:
train_path = r"/content/Training"
test_path = r"/content/Validation"

In [28]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [29]:
for layer in vgg16.layers:
    layer.trainable = False

In [30]:
folders = glob(r'/content/Training/*')

In [31]:
folders

['/content/Training/male', '/content/Training/female']

In [32]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [33]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [34]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [35]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [36]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [37]:
training_set = train_datagen.flow_from_directory(r"/content/Training",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 47009 images belonging to 2 classes.


In [38]:
test_set = test_datagen.flow_from_directory(r"//content/Validation",
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 11649 images belonging to 2 classes.


In [39]:
len(training_set)

1470

In [40]:
len(test_set)

365

In [41]:
model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=100,
  validation_steps=65
)

Epoch 1/20
100/100 [==============================] - 66s 652ms/step - loss: 0.4153 - accuracy: 0.8119 - val_loss: 0.2422 - val_accuracy: 0.9077
Epoch 2/20
100/100 [==============================] - 65s 650ms/step - loss: 0.2284 - accuracy: 0.9125 - val_loss: 0.1830 - val_accuracy: 0.9274
Epoch 3/20
100/100 [==============================] - 65s 649ms/step - loss: 0.2256 - accuracy: 0.9141 - val_loss: 0.2318 - val_accuracy: 0.9091
Epoch 4/20
100/100 [==============================] - 65s 654ms/step - loss: 0.2251 - accuracy: 0.9109 - val_loss: 0.3301 - val_accuracy: 0.8755
Epoch 5/20
100/100 [==============================] - 65s 653ms/step - loss: 0.2053 - accuracy: 0.9234 - val_loss: 0.1727 - val_accuracy: 0.9442
Epoch 6/20
100/100 [==============================] - 64s 640ms/step - loss: 0.2002 - accuracy: 0.9253 - val_loss: 0.1659 - val_accuracy: 0.9298
Epoch 7/20
100/100 [==============================] - 64s 639ms/step - loss: 0.2401 - accuracy: 0.9178 - val_loss: 0.1458 - val_ac

In [42]:
model.save("Gender_vgg16.h5")